In [1]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import time

import sys
sys.path.append('../MoitraRohatgi/')
sys.path.append('../')
import auditor_tools
import algorithms
import experiments
import examples
import our_experiments

Our data is downloaded from https://docs.google.com/uc?id=10h_5og14wbNHU-lapQaS1W6SBdzI7W6Z&export=download"), corresponding to Carl Bauer's Applied Causal Analysis (with R).

### Preprocessing

To create a clean instance, we drop all stores where there is a NaN in either employment before or after; this means our regression is on a slightly different data-set than the one by Card & Krueger (recall that this only serves as a proof-of-concept). The resulting lists contain the delta-values (see writeup), appropriately ordered (decreasing/increasing).

As we create these tables, we also match the data in Table 2 of Card and Krueger
(https://davidcard.berkeley.edu/papers/njmin-aer.pdf).

In [2]:
delta_pa, delta_nj, df2 = our_experiments.LoadCardKruegerData()

NJ mean numbers
x_co_owned                    0.341390
x_southern_nj                 0.280967
x_central_nj                  0.190332
x_northeast_philadelphia      0.000000
x_easton_philadelphia         0.000000
x_st_wage_before              4.612134
x_st_wage_after               5.080849
x_hrs_open_weekday_before    14.418429
x_hrs_open_weekday_after     14.419782
y_ft_employment_before       20.439408
y_ft_employment_after        21.027429
d_nj                          1.000000
d_pa                          0.000000
x_burgerking                  0.410876
x_kfc                         0.205438
x_roys                        0.247734
x_wendys                      0.135952
x_closed_permanently          0.015106
dtype: float64
PA mean numbers
x_co_owned                    0.354430
x_southern_nj                 0.000000
x_central_nj                  0.000000
x_northeast_philadelphia      0.455696
x_easton_philadelphia         0.544304
x_st_wage_before              4.630132
x_st_wage_after  

The 2.75 is close to the coefficient in FTE is close to (but not exactly) the change detected by Card & Krueger (2.76) with the difference likely due to  the fact that we dropped stores where the row contained a NaN for FTE in either before or after. We now want to detect how many stores we need to remove the before/after samples from to flip the 2.75 sign.

In [3]:
start = time.time()
auditor_tools.solve_diff_in_diff(delta_pa,delta_nj)
print('Total time: ',str(time.time()-start)[:5]+'s')

Number of observation pairs to remove:  10
treated removed:  []
untreated removed:  [-18.0, -18.0, -18.5, -18.5, -20.0, -21.5, -29.0, -41.5]
Total time:  0.000s


### This means the coefficient flips if we keep all NJ stores and remove the 10 PA stores with the smallest $\Delta$.
Running the regression with those 10 stores removed, we find that the OLS solution indeed has the sign of the diff-in-diff estimator flip. 

In [4]:
X,Y=our_experiments.LoadKruegerDataWith10PAStoresRemoved()
algorithms.ols(X,Y,np.ones(len(X))) 

array([20.17692308,  0.25365945,  0.72692308, -0.26025641])

This worked well; now, let's try the same in Gurobi.

### To run Gurobi on the same instance, we need to make sure we have the observation pairs that are jointly kept/dropped.

In [5]:
# First we put the data back in the right format and create a list of pairs that have to get the same weight
# i.e., each store has either its before and its after removed or neither
data_X, data_Y = [], []
counter = 0
pairs = []
for x in df2.index:
    # Dummy for whether in NJ
    NJ = 0 if df2.d_pa[x] else 1
    # 1 for intercept, dummy for NJ, dummy for treatment and dummy for treatment*NJ
    data_X.append([1,NJ, 0, 0])
    counter+=1
    data_Y.append(df2.y_ft_employment_before[x])
    data_X.append([1,NJ, 1, NJ])
    data_Y.append(df2.y_ft_employment_after[x])
    pairs.append([counter-1,counter])
    counter+=1
X=np.array(data_X)
Y=np.array(data_Y)

We then run Gurobi with those weights constrained to be equal (for the respective pairs) to show that it does not solve to optimality even given ample time.

In [6]:
start = time.time()
sol=auditor_tools.solve_regression_fractional(X,Y,intercept=False, time_limit=1500, 
                                              pairs=pairs)
print('Current time: ',str(time.time()-start)[:5]+'s')
sol_int=auditor_tools.solve_regression_integral(X,Y, intercept=False, time_limit=1500,
                             warm_start=[1 if sol[-2][x].X>.999 else 0 for x in 
                                         range(len(sol[-2]))],
                                 warm_start_ub=sol[0],pairs=pairs)
print('Total time: ',str(time.time()-start)[:5]+'s')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-04
set residual constraints
Set parameter NonConvex to value 2
Set parameter TimeLimit to value 150
start solving
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1920 rows, 771 columns and 2304 nonzeros
Model fingerprint: 0xcc0ddf0a
Model has 4 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [5e+00, 8e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1536 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective -0.0000000
Presolve removed 1920 rows and 384 columns
Presolve time: 0.01s
Presolved: 4312 rows, 1465 columns, 13999 nonzeros
Presolved model has 1077 bilinear constraint(s

We can observe two features from running on Gurobi on this instance: first, even significant time Gurobi does not manage to certify the optimal solution for this ''easy'' instance (easy in the sense that the exact algorithm finds it in .001s); second, Gurobi does identify the optimal solution (basically) instantaneously, it just cannot find a certificate.